# UNIQLO Deal Research
Deep analysis of deal patterns, timing, seasonality, and price predictions.

In [21]:
import sys
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

from analysis.queries import (
    get_engine, load_timeseries, load_price_history,
    load_deal_heatmap, load_seasonal, load_top_products
)
from analysis.features import engineer_features
from analysis.predictions import predict_price, deal_probability, price_drop_probability, best_time_to_buy, price_drop_timing

engine = get_engine()
print('Connected to Supabase')

Connected to Supabase


## 1. Load Data

In [22]:
import os

# --- CONFIG ---
FILTER_SIZE   = None #['XS', 'S', 'M', 'L', 'XL']  # set to None to see all sizes
FILTER_GENDER = None      # 'woman', 'man', or None
DAYS_BACK     = None      # last N days, or None for all history
FORCE_RELOAD  = False     # set True to re-fetch from DB and overwrite cache

# Build a cache filename that reflects the active filters
_size_tag   = '-'.join(FILTER_SIZE) if isinstance(FILTER_SIZE, list) else (FILTER_SIZE or 'all')
_gender_tag = FILTER_GENDER or 'all'
_days_tag   = str(DAYS_BACK) if DAYS_BACK else 'all'
CACHE_PATH  = f'cache/raw_{_size_tag}_{_gender_tag}_{_days_tag}.csv'

os.makedirs('cache', exist_ok=True)

if not FORCE_RELOAD and os.path.exists(CACHE_PATH):
    df_raw = pd.read_csv(CACHE_PATH)
    df_raw['fetched_at'] = pd.to_datetime(df_raw['fetched_at'], format='ISO8601')
    df_raw['date']       = pd.to_datetime(df_raw['date'])
    print(f'Loaded from cache: {CACHE_PATH}')
else:
    df_raw = load_timeseries(engine, size=FILTER_SIZE, gender=FILTER_GENDER, days=DAYS_BACK)
    df_raw.to_csv(CACHE_PATH, index=False)
    print(f'Fetched from DB and saved to: {CACHE_PATH}')

df = engineer_features(df_raw)

print(f'Rows: {len(df):,}')
print(f'Date range: {df["fetched_at"].min().date()} → {df["fetched_at"].max().date()}')
print(f'Unique products: {df["product_id"].nunique()}')
print(f'Sizes: {sorted(df["size"].unique())}')
df.head(3)

Loaded from cache: cache/raw_all_all_all.csv
Rows: 58,218
Date range: 2025-04-09 → 2026-02-20
Unique products: 1631
Sizes: ['22inch', '23inch', '24inch', '25inch', '26inch', '27inch', '28inch', '29inch', '30inch', '31inch', '32inch', '33inch', '34inch', '35inch', '36inch', '37', '37-41.5', '39-42', '3XL', '42-46', '42-46(27-29cm)', '65/70 D E', '65/70 F G', '75/80 B C', '75/80 D E', '75/80 F G', '85/90 AA A', '85/90 D E', '85/90 F G', 'L', 'M', 'One Size', 'S', 'Unavailable', 'Unknown', 'XL', 'XS', 'XXL', 'XXS']


,id,fetched_at,date,day_of_week,hour,month,week,year,promo_price,original_price,...,size,name,product_id,gender,day_name,month_name,season,is_weekend,is_good_deal,days_since_start
0,1a8e8aa8-7511-47b9-bb68-b12ba4b22111,2025-04-09 19:24:18.775000+00:00,2025-04-09,3,19,4,15,2025,14.9,24.9,...,Unknown,Krepp Jersey BH-Trägertop,E477052-000,unknown,Wed,Apr,Spring,False,False,0.0
1,8aeb9c43-eb3d-4fce-b116-05e5f6c9c5e7,2025-04-09 19:24:18.775000+00:00,2025-04-09,3,19,4,15,2025,12.9,19.9,...,Unknown,PEANUTS Vintage Books UT T-Shirt,E474450-000,unknown,Wed,Apr,Spring,False,False,0.0
2,4b36b938-ecc9-482d-a1e1-41f01b5cf1cd,2025-04-09 19:24:18.775000+00:00,2025-04-09,3,19,4,15,2025,12.9,19.9,...,Unknown,PEANUTS Memories of Snoopy UT T-Shirt,E476605-000,unknown,Wed,Apr,Spring,False,False,0.0


## 2. When Do Prices Actually Drop? (Day × Hour Heatmap)

In [23]:
drop_rate, avg_drop = price_drop_timing(df)

fig = make_subplots(rows=1, cols=2,
                    subplot_titles=['Drop Frequency (how often price falls)',
                                    'Avg Drop Size when it does fall (%)'])

fig.add_heatmap(z=drop_rate.values,
                x=drop_rate.columns.tolist(),
                y=[f'{h:02d}:00' for h in drop_rate.index],
                colorscale='Inferno',
                text=np.round(drop_rate.values * 100, 1),
                texttemplate='%{text}%',
                hovertemplate='%{x} %{y}<br>Drop rate: %{z:.1%}<extra></extra>',
                row=1, col=1)

fig.add_heatmap(z=avg_drop.values,
                x=avg_drop.columns.tolist(),
                y=[f'{h:02d}:00' for h in avg_drop.index],
                colorscale='Inferno',
                text=np.round(avg_drop.values, 1),
                texttemplate='%{text}%',
                hovertemplate='%{x} %{y}<br>Avg drop: %{z:.1f}%<extra></extra>',
                row=1, col=2)

fig.update_layout(
    title=f'Price Drop Map — Size: {FILTER_SIZE or "all"}, Gender: {FILTER_GENDER or "all"}',
    height=600
)
fig.show()

best = best_time_to_buy(drop_rate)
print(f"Most likely price drop: {best['best_day']} at {best['best_hour']:02d}:00 → {best['probability']*100:.1f}% of fetches show a price decrease")

best = best_time_to_buy(drop_rate)

# Flatten drop_rate into ranked day+hour combinations
flat = (
    drop_rate.stack()
    .reset_index()
)
flat.columns = ['hour', 'day', 'probability']
flat['label'] = flat['day'] + ' ' + flat['hour'].apply(lambda h: f'{h:02d}:00')
flat = flat.sort_values('probability', ascending=True).tail(20)

is_best = (flat['day'] == best['best_day']) & (flat['hour'] == best['best_hour'])
colors = ['#e74c3c' if b else '#3498db' for b in is_best]

fig_best = go.Figure(go.Bar(
    x=flat['probability'] * 100,
    y=flat['label'],
    orientation='h',
    marker_color=colors,
    text=[f"{p*100:.1f}%" for p in flat['probability']],
    textposition='outside',
    hovertemplate='%{y}<br>Drop probability: %{x:.1f}%<extra></extra>'
))
fig_best.update_layout(
    title=f'Top 20 Best Times to Check for Price Drops — Best: {best["best_day"]} {best["best_hour"]:01d}:00 ({best["probability"]*100:.1f}%)',
    xaxis_title='Price Drop Probability (%)',
    height=600,
    margin=dict(l=150)
)
fig_best.show()

Most likely price drop: Wed at 05:00 → 90.0% of fetches show a price decrease


## 3. Seasonal: weekly/monthly components

In [24]:
seasonal_df = (
    df.groupby(['year', 'month', 'week'])
    .agg(
        observations=('id', 'count'),
        good_deals=('is_good_deal', 'sum'),
        avg_discount=('discount_percent', 'mean'),
        min_price=('promo_price', 'min'),
    )
    .reset_index()
)

# --- Weekly ---
weekly = (
    seasonal_df.groupby(['year', 'week'])
    .agg(good_deals=('good_deals','sum'), observations=('observations','sum'))
    .reset_index()
)
weekly['deal_rate'] = weekly['good_deals'] / weekly['observations']
weekly['date'] = pd.to_datetime(
    weekly['year'].astype(str) + '-W' + weekly['week'].astype(str).str.zfill(2) + '-4',
    format='%G-W%V-%u'
)
weekly = weekly.sort_values('date')

fig_w = go.Figure(go.Bar(
    x=weekly['date'], y=weekly['deal_rate'],
    marker_color='#2ecc71',
    hovertemplate='%{x}<br>Deal Rate: %{y:.1%}<extra></extra>'
))
fig_w.update_layout(title='Deal Rate — Weekly', yaxis_tickformat='.0%', height=400)
fig_w.show()

# --- Monthly ---
monthly = (
    seasonal_df.groupby(['year', 'month'])
    .agg(good_deals=('good_deals','sum'), observations=('observations','sum'))
    .reset_index()
)
monthly['deal_rate'] = monthly['good_deals'] / monthly['observations']
monthly['date'] = pd.to_datetime(monthly[['year','month']].assign(day=1))
monthly = monthly.sort_values('date')
monthly['label'] = monthly['date'].dt.strftime('%b %Y')

fig_m = go.Figure(go.Bar(
    x=monthly['label'], y=monthly['deal_rate'],
    marker_color='#2ecc71',
    text=monthly['deal_rate'].map(lambda x: f'{x:.0%}'),
    textposition='outside',
    hovertemplate='%{x}<br>Deal Rate: %{y:.1%}<extra></extra>'
))
fig_m.update_layout(title='Deal Rate — Monthly', yaxis_tickformat='.0%', height=400)
fig_m.show()

# --- By season ---
season_map = {12:'Winter',1:'Winter',2:'Winter',3:'Spring',4:'Spring',5:'Spring',
              6:'Summer',7:'Summer',8:'Summer',9:'Autumn',10:'Autumn',11:'Autumn'}
monthly['season'] = monthly['month'].map(season_map)
season_agg = monthly.groupby('season')['deal_rate'].mean().reset_index()

fig_s = px.bar(season_agg, x='season', y='deal_rate',
               color='deal_rate', color_continuous_scale='Greens',
               text=season_agg['deal_rate'].map(lambda x: f'{x:.0%}'),
               title='Deal Rate by Season',
               category_orders={'season':['Winter','Spring','Summer','Autumn']})
fig_s.update_layout(height=400, yaxis_tickformat='.0%', showlegend=False, coloraxis_showscale=False)
fig_s.show()

In [25]:
ACTION_COLORS = {
    'SUPER':          '#27ae60',
    'GOOD DEAL':      '#2ecc71',
    'BIG DISCOUNT':   '#f39c12',
    'VERY CHEAP':     '#f1c40f',
    'CHEAP UPPER MID':'#1abc9c',
    'DECENT':         '#3498db',
    'NEUTRAL':        '#95a5a6',
    'AVOID':          '#e74c3c',
}
ACTION_ORDER = list(ACTION_COLORS.keys())

def action_stacked_bar(group_col, label_col=None, title='', category_orders=None):
    grp = (
        df.groupby([group_col, 'action'])
        .size()
        .reset_index(name='count')
    )
    total = grp.groupby(group_col)['count'].transform('sum')
    grp['pct'] = grp['count'] / total

    fig = go.Figure()
    for action in ACTION_ORDER:
        sub = grp[grp['action'] == action]
        x = sub[label_col or group_col] if label_col else sub[group_col]
        fig.add_trace(go.Bar(
            x=x,
            y=sub['pct'],
            name=action,
            marker_color=ACTION_COLORS[action],
            hovertemplate=f'{action}<br>%{{x}}<br>%{{y:.1%}}<extra></extra>'
        ))

    fig.update_layout(
        barmode='stack',
        title=title,
        yaxis_tickformat='.0%',
        yaxis_title='Share of observations',
        height=400,
        legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='left', x=0),
        **(dict(xaxis=dict(categoryorder='array', categoryarray=category_orders)) if category_orders else {})
    )
    fig.show()

# --- By week ---
df['week_label'] = pd.to_datetime(
    df['year'].astype(str) + '-W' + df['week'].astype(str).str.zfill(2) + '-4',
    format='%G-W%V-%u'
).dt.strftime('W%V %Y')
week_order = df.sort_values(['year','week'])['week_label'].unique().tolist()
action_stacked_bar('week_label', title='Deal Actions by Week',
                   category_orders=week_order)

# --- By month ---
df['month_label'] = pd.to_datetime(df[['year','month']].assign(day=1)).dt.strftime('%b %Y')
month_order = df.sort_values(['year','month'])['month_label'].unique().tolist()
action_stacked_bar('month_label', title='Deal Actions by Month',
                   category_orders=month_order)

# --- By season ---
action_stacked_bar('season', title='Deal Actions by Season',
                   category_orders=['Winter','Spring','Summer','Autumn'])

In [26]:
# Calculate discount from original → promo price
disc = df.copy()
disc['discount_pct'] = (disc['original_price'] - disc['promo_price']) / disc['original_price'] * 100
disc = disc[disc['original_price'] > 0]   # drop rows with no original price

# --- Overall histogram ---
fig_hist = go.Figure(go.Histogram(
    x=disc['discount_pct'],
    nbinsx=50,
    marker_color='#3498db',
    opacity=0.85,
    hovertemplate='Discount: %{x:.1f}%<br>Count: %{y}<extra></extra>'
))
fig_hist.update_layout(
    title='Distribution of Discounts (original → promo price)',
    xaxis_title='Discount (%)',
    yaxis_title='Number of observations',
    height=400,
)
fig_hist.show()

# --- By action (overlapping histograms, normalised) ---
ACTION_COLORS = {
    'SUPER':          '#27ae60',
    'GOOD DEAL':      '#2ecc71',
    'BIG DISCOUNT':   '#f39c12',
    'VERY CHEAP':     '#f1c40f',
    'CHEAP UPPER MID':'#1abc9c',
    'DECENT':         '#3498db',
    'NEUTRAL':        '#95a5a6',
    'AVOID':          '#e74c3c',
}

fig_act = go.Figure()
for action, color in ACTION_COLORS.items():
    sub = disc[disc['action'] == action]['discount_pct'].dropna()
    if len(sub) < 5:
        continue
    fig_act.add_trace(go.Histogram(
        x=sub,
        name=action,
        nbinsx=40,
        histnorm='probability density',
        marker_color=color,
        opacity=0.6,
        #hovertemplate=f'{action}<br>Discount: %{{x:.1f}%}}<extra></extra>'
    ))
fig_act.update_layout(
    barmode='overlay',
    title='Discount Distribution by Action (normalised)',
    xaxis_title='Discount (%)',
    yaxis_title='Density',
    height=450,
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='left', x=0),
)
fig_act.show()

# --- Box plot per action (ordered by median discount) ---
action_order = (
    disc.groupby('action')['discount_pct']
    .median()
    .sort_values(ascending=False)
    .index.tolist()
)
fig_box = go.Figure()
for action in action_order:
    sub = disc[disc['action'] == action]['discount_pct'].dropna()
    if len(sub) < 5:
        continue
    fig_box.add_trace(go.Box(
        y=sub,
        name=action,
        marker_color=ACTION_COLORS.get(action, '#aaa'),
        boxmean=True,
        hovertemplate=f'{action}<br>Discount: %{{y:.1f}}%<extra></extra>'
    ))
fig_box.update_layout(
    title='Discount Spread by Action (ordered by median)',
    yaxis_title='Discount (%)',
    height=450,
    showlegend=False,
)
fig_box.show()

In [27]:
DAY_ORDER = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

# reuse disc from previous cell
disc['week_label'] = pd.to_datetime(
    disc['year'].astype(str) + '-W' + disc['week'].astype(str).str.zfill(2) + '-4',
    format='%G-W%V-%u'
).dt.strftime('W%V %Y')
week_order = disc.sort_values(['year', 'week'])['week_label'].unique().tolist()

# --- By day of week ---
fig_day = go.Figure()
for day in DAY_ORDER:
    sub = disc[disc['day_name'] == day]['discount_pct'].dropna()
    fig_day.add_trace(go.Box(
        y=sub,
        name=day,
        marker_color='#3498db',
        boxmean=True,
        hovertemplate=f'{day}<br>Discount: %{{y:.1f}}%<extra></extra>'
    ))
fig_day.update_layout(
    title='Discount Distribution by Day of Week',
    yaxis_title='Discount (%)',
    height=400,
    showlegend=False,
)
fig_day.show()

# --- By week ---
fig_week = go.Figure()
for wk in week_order:
    sub = disc[disc['week_label'] == wk]['discount_pct'].dropna()
    fig_week.add_trace(go.Box(
        y=sub,
        name=wk,
        marker_color='#9b59b6',
        boxmean=False,
        hovertemplate=f'{wk}<br>Discount: %{{y:.1f}}%<extra></extra>'
    ))
fig_week.update_layout(
    title='Discount Distribution by Week',
    yaxis_title='Discount (%)',
    xaxis=dict(categoryorder='array', categoryarray=week_order),
    height=450,
    showlegend=False,
)
fig_week.show()

## Coats & Warm Clothing — Biggest Price Drops

In [43]:
OUTERWEAR_KEYWORDS = [
    'mantel', 'coat',
    'jacke', 'jacket',
    'parka',
    'blouson',
    'anorak',
    'daunenjacke',
    'daunenmantel',
    'daunenweste',
    'daunenparka',
    'trenchcoat', 'trench',
    'blocktech',
    'puffertech', 'puffer',
    'hybrid down',
    'ultra light down', 'uld',
    'seamless down',
    'powder soft', 'teddyfleece'
    'ultra warm', 'fleece', 'teddy', 'soufflé', 'souffle'
]

warm_disc = df.copy()
sizes_to_keep = ['M', 'L', 'XL']  # adjust as needed
warm_disc = warm_disc[warm_disc['size'].isin(sizes_to_keep)]



In [44]:
warm_disc['name'].unique()

array(['HEATTECH Extra Warm Baumwolle Thermo Langarmshirt',
       '100 % Leinen T-Shirt (Rundhals)',
       'HEATTECH Extra Warm Baumwolle Thermo Langarmshirt (weiter Ausschnitt)',
       'SpongeBob SquarePants Cactus Plant Flea Market UT Hoodie',
       'KAWS + Warhol UT Sweatshirt',
       'HEATTECH Thermo Langarmshirt (weiter Rundhals)',
       'Soufflé Yarn Strickpullover (Rundhals, Colourblocking)',
       '100 % Merinowolle Rollkragenpullover (gerippt)',
       'Ultra Stretch AIRism Jogginghose', 'AIRism Baumwolle T-Shirt',
       'AIRism Leggings (UV-Schutz, Seitentasche)',
       'Wollmix Wickelmantel', '100 % Premium Leinen Bluse',
       'SMART 7/8 Hose',
       'HEATTECH Fleece Thermo Langarmshirt (Rollkragen)',
       'Lammwolle Pullover (Rundhals)', 'Chinohose (Wide Fit)',
       'AIRism Shorts (soft)', 'Jeans (Wide Fit)',
       'Soufflé Yarn Strickkleid',
       'HEATTECH Extra Warm Baumwolle Thermo Leggings',
       'KAWS + Warhol UT Jogginghose',
       'HEATTECH Flee

In [52]:
OUTERWEAR_KEYWORDS = [
    'mantel', 'coat',
    'jacke', 'jacket',
    'parka',
    'blouson',
    'anorak',
    'daunenjacke',
    'daunenmantel',
    'daunenweste',
    'daunenparka',
    'trenchcoat', 'trench',
    'blocktech',
    'puffertech', 'puffer',
    'hybrid down',
    'ultra light down', 
    'seamless down',
    'powder soft', 'teddyfleece',
    'ultra warm', 'fleece', 'teddy', 'soufflé', 'souffle'
]

warm_disc = df.copy()
sizes_to_keep = ['M', 'L', 'XL']  # adjust as needed
warm_disc = warm_disc[warm_disc['size'].isin(sizes_to_keep)]


# Ensure fetched_at is datetime
warm_disc['date'] = pd.to_datetime(warm_disc['date'])

# Filter by last 7 days
cutoff_date = pd.Timestamp.now() - pd.Timedelta(days=7)
warm_disc = warm_disc[warm_disc['date'] >= cutoff_date]

warm_disc['discount_pct'] = (
    (warm_disc['original_price'] - warm_disc['promo_price']) / warm_disc['original_price'] * 100
)
warm_disc = warm_disc[warm_disc['original_price'] > 0]

pattern = '|'.join(OUTERWEAR_KEYWORDS)
warm = warm_disc[warm_disc['name'].str.contains(pattern, case=False, na=False)]
warm_deals = warm[warm['discount_pct'] > 65]

# Best deal ever seen per product × size
best_warm = (
    warm_deals
    .sort_values('discount_pct', ascending=False)
    .groupby(['product_id', 'size'])
    .agg(
        name=('name', 'first'),
        gender=('gender', 'first'),
        original_price=('original_price', 'max'),
        best_promo=('promo_price', 'min'),
        max_discount=('discount_pct', 'max'),
        action=('action', 'first'),
        last_seen=('fetched_at', 'max'),
    )
    .reset_index()
    .sort_values('max_discount', ascending=False)
)

print(f"Outerwear items with >75% discount: "
      f"{len(best_warm)} product-size combos | {best_warm['product_id'].nunique()} unique products")

# --- Bar chart: top items by max discount ---
top_n = best_warm.head(30).copy()
top_n['label'] = top_n['name'].str[:40] + ' [' + top_n['size'] + ']'
top_n = top_n.sort_values('max_discount', ascending=True)

fig_warm = go.Figure(go.Bar(
    x=top_n['max_discount'],
    y=top_n['label'],
    orientation='h',
    marker_color='#2980b9',
    text=[f"{v:.0f}%  ({r:.2f}€ → {p:.2f}€)"
          for v, r, p in zip(top_n['max_discount'], top_n['original_price'], top_n['best_promo'])],
    textposition='outside',
    hovertemplate='%{y}<br>Max discount: %{x:.1f}%<extra></extra>',
))
fig_warm.update_layout(
    title='Top 30 Warm Clothing Items with >75% Discount (best deal ever seen)',
    xaxis_title='Max Discount (%)',
    xaxis=dict(range=[0, top_n['max_discount'].max() * 2]),
    height=750,
    margin=dict(l=350, r=200),
)
fig_warm.show()

# # --- Table view ---
# display(
#     best_warm[['name', 'size', 'gender', 'original_price', 'best_promo', 'max_discount', 'action', 'last_seen']]
#     .rename(columns={
#         'original_price': 'orig_€',
#         'best_promo': 'promo_€',
#         'max_discount': 'discount_%',
#         'last_seen': 'last_seen_at',
#     })
#     .assign(**{'discount_%': lambda d: d['discount_%'].round(1),
#                'orig_€': lambda d: d['orig_€'].round(2),
#                'promo_€': lambda d: d['promo_€'].round(2)})
#     .reset_index(drop=True)
# )

Outerwear items with >75% discount: 23 product-size combos | 12 unique products


In [35]:
warm_disc.columns

Index(['id', 'fetched_at', 'date', 'day_of_week', 'hour', 'month', 'week',
       'year', 'promo_price', 'original_price', 'discount_percent', 'rating',
       'reviews', 'action', 'color', 'size', 'name', 'product_id', 'gender',
       'day_name', 'month_name', 'season', 'is_weekend', 'is_good_deal',
       'days_since_start', 'week_label', 'month_label'],
      dtype='object')

## 10. Summary Stats

In [28]:
good = df[df['is_good_deal']]

print('=== SUMMARY ===')
print(f"Total observations:      {len(df):,}")
print(f"Good deals:              {len(good):,} ({len(good)/len(df)*100:.1f}%)")
print(f"Unique products:         {df['product_id'].nunique()}")
print(f"Avg discount (all):      {df['discount_percent'].mean():.1f}%")
print(f"Avg discount (deals):    {good['discount_percent'].mean():.1f}%")
print(f"Cheapest price seen:     {df['promo_price'].min():.2f}€")
print(f"\nTop actions:")
print(df['action'].value_counts().to_string())
print(f"\nTop seasons for deals:")
print(good.groupby('season')['is_good_deal'].count().sort_values(ascending=False).to_string())

=== SUMMARY ===
Total observations:      58,218
Good deals:              16,562 (28.4%)
Unique products:         1631
Avg discount (all):      57.0%
Avg discount (deals):    72.9%
Cheapest price seen:     1.00€

Top actions:
action
NEUTRAL              28751
DECENT                5674
GOOD DEAL             5536
CHEAP UPPER MID       5535
SUPER                 5330
WAIT FOR SALE         3166
CHEAP BUT MID         1951
AVOID                 1759
STEAL                  104
BIG DISCOUNT            82
VERY CHEAP              79
LOW QUALITY             49
OK DEAL                 43
OK                      37
QUALITY PICK            35
FAIR DEAL               30
GREAT DEAL              23
SKIP                    18
TOP BUT EXPENSIVE       12
BARGAIN BIN              4

Top seasons for deals:
season
Summer    5291
Winter    4305
Autumn    3775
Spring    3191
